In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#Fetch the table with a combination of Request and BeautifolSoup
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers).text
soup = BeautifulSoup(req, 'xml')
# Only fetch the table we are interested in
html_table = soup.find('table',{'class': 'wikitable sortable'})


html_table_rows = html_table.find_all('tr')
table_data = []
#Loop over the table itself to make sure we know the procedure of getting the information
for table_row in html_table_rows:
    cells = []
    for table_cell in table_row.find_all('td'):
        cells.append(table_cell.text.strip())
    table_data.append(cells)

# Transfer to a dataframe with the given columns
table_df = pd.DataFrame(table_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

# Remove any rows with an empty Borough or it is not assigned
table_df = table_df[~table_df['Borough'].isnull()]
table_df.drop(table_df[table_df.Borough == 'Not assigned'].index, inplace=True)
table_df.reset_index(drop = True, inplace = True)

# Make sure we do the grouping as given, with neighbourhoods seperated by comma
table_df = table_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
table_df['Neighbourhood'].replace('Not assigned', table_df['Borough'], inplace=True)

# Presto.gif
table_df.shape

(103, 3)